In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-09-01T11:06:47.689947-04:00

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.28.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.11.0-37-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit



In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import sklearn
import gc
import os
import lightgbm

In [6]:
%watermark --iversions

pandas  : 1.4.3
numpy   : 1.22.4
lightgbm: 3.3.2
sklearn : 1.1.2



In [7]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

In [8]:
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,147,252,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
X = train[train.columns[1:]].values
X_test = test[test.columns[1:]].values
Y = train.label.values

In [11]:
train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

(60000, 10)

In [27]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = LGBMClassifier(n_estimators=400)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold accuracy: 0.9841666666666666
Fitting fold 2
Fold accuracy: 0.98125
Fitting fold 3
Fold accuracy: 0.9819166666666667
Fitting fold 4
Fold accuracy: 0.9828333333333333
Fitting fold 5
Fold accuracy: 0.9781666666666666
CPU times: user 5h 51min 43s, sys: 1min 39s, total: 5h 53min 22s
Wall time: 12min


In [25]:
0.9841666666666666

0.9841666666666666

In [28]:
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

0.9816666666666667


In [29]:
0.9775833333333334

0.9775833333333334

In [30]:
preds = np.argmax(test_preds, axis=1)
preds

array([7, 2, 1, ..., 4, 5, 6])

In [31]:
test.label.values

array([7, 2, 1, ..., 4, 5, 6])

In [32]:
accuracy_score(test.label.values, preds)

0.9825

In [36]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = LGBMClassifier(n_estimators=400,random_state=137, 
                                           max_depth=15)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.984
Fitting fold 2
Fold accuracy: 0.9810833333333333
Fitting fold 3
Fold accuracy: 0.9815
Fitting fold 4
Fold accuracy: 0.9826666666666667
Fitting fold 5
Fold accuracy: 0.9786666666666667
0.9815833333333334


NameError: name 'Y_test' is not defined

In [37]:
accuracy_score(test.label.values, preds)

0.9825

In [38]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = LGBMClassifier(n_estimators=400,random_state=137, 
                                           max_depth=12)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9843333333333333
Fitting fold 2
Fold accuracy: 0.9815833333333334
Fitting fold 3
Fold accuracy: 0.98125
Fitting fold 4
Fold accuracy: 0.9818333333333333
Fitting fold 5
Fold accuracy: 0.9783333333333334
0.9814666666666667
0.9825
CPU times: user 5h 48min 25s, sys: 1min 41s, total: 5h 50min 6s
Wall time: 11min 54s


In [39]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = LGBMClassifier(n_estimators=400,random_state=137, subsample=0.9)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9841666666666666
Fitting fold 2
Fold accuracy: 0.98125
Fitting fold 3
Fold accuracy: 0.9819166666666667
Fitting fold 4
Fold accuracy: 0.9828333333333333
Fitting fold 5
Fold accuracy: 0.9781666666666666
0.9816666666666667
0.9825
CPU times: user 5h 51min 53s, sys: 1min 40s, total: 5h 53min 34s
Wall time: 12min 2s


In [40]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = LGBMClassifier(n_estimators=400,random_state=137, colsample_bytree=0.9,
                           subsample=0.9)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9843333333333333
Fitting fold 2
Fold accuracy: 0.9814166666666667
Fitting fold 3
Fold accuracy: 0.9816666666666667
Fitting fold 4
Fold accuracy: 0.9818333333333333
Fitting fold 5
Fold accuracy: 0.9785
0.98155
0.9825
CPU times: user 5h 28min 16s, sys: 1min 29s, total: 5h 29min 46s
Wall time: 11min 4s


In [41]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = LGBMClassifier(n_estimators=400,random_state=137, colsample_bytree=0.9,
                           subsample=0.9, max_depth=12)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

print(accuracy_score(test.label.values, preds))

Fitting fold 1
Fold accuracy: 0.9845833333333334
Fitting fold 2
Fold accuracy: 0.9815
Fitting fold 3
Fold accuracy: 0.9813333333333333
Fitting fold 4
Fold accuracy: 0.9828333333333333
Fitting fold 5
Fold accuracy: 0.9778333333333333
0.9816166666666667
0.9825
CPU times: user 5h 13min 5s, sys: 1min 20s, total: 5h 14min 25s
Wall time: 10min 26s


In [42]:
preds = np.argmax(test_preds, axis=1)

In [43]:
print(accuracy_score(test.label.values, preds))

0.9828
